# AR1 Pulsar Observation report

## Files to be processed

In [ ]:
directory = "/data/"

## Set up environment

In [ ]:
%pylab inline

In [ ]:
import psrchive as psr
import os

## Load data

In [ ]:
# Load files with *.ar into the list.
files = []
for file in os.listdir(directory):
    if file.endswith(".ar"):
        files.append(file)
files.sort()
# Build a list of objects, each holding individual archive.
archives = []
archives = [psr.Archive_load(directory + file) for file in files]
# Append all files to a first archive.
print archives[0]
for i in range(1, len(archives)):
    print archives[i]
    archives[0].append(archives[i])

In [ ]:
print "nbin             Number of pulse phase bins                 %s" % archives[0].get_nbin()
print "nchan            Number of frequency channels               %s" % archives[0].get_nchan()
print "npol             Number of polarizations                    %s" % archives[0].get_npol()
print "nsubint          Number of sub-integrations                 %s" % archives[0].get_nsubint()
print "type             Observation type                           %s" % archives[0].get_type()
print "site             Telescope name                             %s" % archives[0].get_telescope()
print "name             Source name                                %s" % archives[0].get_source()
RA = archives[0].get_coordinates().ra()
Dec = archives[0].get_coordinates().dec()
print "coord            Source coordinates                         %s%s" % (RA.getHMS(), Dec.getDMS())
print "freq             Centre frequency (MHz)                     %s" % archives[0].get_centre_frequency()
print "bw               Bandwidth (MHz)                            %s" % archives[0].get_bandwidth()
print "dm               Dispersion measure (pc/cm^3)               %s" % archives[0].get_dispersion_measure()
print "rm               Rotation measure (rad/m^2)                 %s" % archives[0].get_rotation_measure()
print "dmc              Dispersion corrected                       %s" % archives[0].get_dedispersed()
print "rmc              Faraday Rotation corrected                 %s" % archives[0].get_faraday_corrected()
print "polc             Polarization calibrated                    %s" % archives[0].get_poln_calibrated()
print "scale            Data units                                 %s" % archives[0].get_scale()
print "state            Data state                                 %s" % archives[0].get_state()
print "length           Observation duration (s)                   %s" % archives[0].integration_length()
print "rcvr:name        Receiver name                              %s" % archives[0].get_receiver_name()
print "rcvr:basis       Basis of receptors                         %s" % archives[0].get_basis()
print "be:name          Name of the backend instrument             %s" % archives[0].get_backend_name()

In [ ]:
profile = archives[0].clone()
profile.dedisperse()
profile.remove_baseline()
profile.centre_max_bin()
profile.tscrunch()
profile.fscrunch()
profile.pscrunch()
profile_data = profile.get_data()
profile_data = profile_data.flatten()

In [ ]:
figure(figsize=[15,10])
plt.title(profile.get_source())
plt.xlabel("Pulse phase (bins)")
plt.ylabel("Flux (a.u.)")
plt.xlim(0, profile.get_nbin() - 1.0)
plt.plot(profile_data)

In [ ]:
bandpass = archives[0].clone()
bandpass.remove_baseline()
bandpass.tscrunch()
bandpass.convert_state("Stokes")
bandpass_data = bandpass.get_data()
bandpass_data = bandpass_data.squeeze()

In [ ]:
figure(figsize=[15,10])
plt.xlim(0, bandpass.get_nchan() - 1.0)
plt.xlabel("Channel number")
plt.ylabel("Flux (a.u.)")
plt.plot(np.average(bandpass_data[0,:,:],axis=1))

In [ ]:
dynamic_Coherence = archives[0].clone()
dynamic_Coherence.dedisperse()
dynamic_Coherence.remove_baseline()
dynamic_Coherence.tscrunch()
dynamic_Coherence.centre_max_bin()
dynamic_Coherence_data = dynamic_Coherence.get_data()
lowFreq = dynamic_Coherence.get_centre_frequency() - dynamic_Coherence.get_bandwidth() / 2.0
highFreq = dynamic_Coherence.get_centre_frequency() + dynamic_Coherence.get_bandwidth() / 2.0

In [ ]:
figure(figsize = [15,10])
plt.subplot(2, 2, 1)
plt.title("XX")
plt.imshow(dynamic_Coherence_data[0, 0, :, :], extent = (0, dynamic_Coherence.get_nbin() - 1.0, lowFreq, highFreq), cmap = get_cmap("binary"))
plt.subplot(2, 2, 2)
plt.title("YY")
plt.imshow(dynamic_Coherence_data[0, 1, :, :], extent = (0, dynamic_Coherence.get_nbin() - 1.0, lowFreq, highFreq), cmap = get_cmap("binary"))
plt.subplot(2, 2, 3)
plt.title("Re(XY)")
plt.imshow(dynamic_Coherence_data[0, 2, :, :], extent = (0, dynamic_Coherence.get_nbin() - 1.0, lowFreq, highFreq), cmap = get_cmap("binary"))
plt.subplot(2, 2, 4)
plt.title("Im(XY)")
plt.imshow(dynamic_Coherence_data[0, 3, :, :], extent = (0, dynamic_Coherence.get_nbin() - 1.0, lowFreq, highFreq), cmap = get_cmap("binary"))

In [ ]:
dynamic_Stokes = archives[0].clone()
dynamic_Stokes.dedisperse()
dynamic_Stokes.remove_baseline()
dynamic_Stokes.tscrunch()
dynamic_Stokes.centre_max_bin()
dynamic_Stokes.convert_state("Stokes")
dynamic_Stokes_data = dynamic_Stokes.get_data()
lowFreq = dynamic_Stokes.get_centre_frequency() - dynamic_Stokes.get_bandwidth() / 2.0
highFreq = dynamic_Stokes.get_centre_frequency() + dynamic_Stokes.get_bandwidth() / 2.0

In [ ]:
figure(figsize = [15,10])
plt.subplot(2, 2, 1)
plt.title("I")
plt.imshow(dynamic_Stokes_data[0, 0, :, :], extent = (0, dynamic_Stokes.get_nbin() - 1.0, lowFreq, highFreq), cmap = get_cmap("binary"))
plt.subplot(2, 2, 2)
plt.title("Q")
plt.imshow(dynamic_Stokes_data[0, 1, :, :], extent = (0, dynamic_Stokes.get_nbin() - 1.0, lowFreq, highFreq), cmap = get_cmap("binary"))
plt.subplot(2, 2, 3)
plt.title("U")
plt.imshow(dynamic_Stokes_data[0, 2, :, :], extent = (0, dynamic_Stokes.get_nbin() - 1.0, lowFreq, highFreq), cmap = get_cmap("binary"))
plt.subplot(2, 2, 4)
plt.title("V")
plt.imshow(dynamic_Stokes_data[0, 3, :, :], extent = (0, dynamic_Stokes.get_nbin() - 1.0, lowFreq, highFreq), cmap = get_cmap("binary"))